## Evaluate GeomLAMA

In [ ]:
# load results

import json 
dataset = "candle"
model = "ChatGPT"
data = json.load(open(f'../results/{dataset}/{model}/results.json'))

In [11]:
def is_exact_match(sample):
    generated = sample['generated']
    gd_answers = sample['answer'].split(', ')

    try:
        # Find the index of the first occurrence of ":"
        colon_index = generated.index(":")
        # Find the index of the first occurrence of "}" after the ":" index
        closing_brace_index = generated.index("}", colon_index)

        # Extract the value between ":" and "}"
        answers = generated[colon_index + 1:closing_brace_index].strip(' "')
        
        answers = answers.replace("\n",'').strip()
        answers = answers.split(', ')
        answers = [answer.strip() for answer in answers]
        # print(gd_answers, answers)
        for answer in answers:
            if answer not in gd_answers:
                # print(False)
                return False
        return True
    except:
        return False
    


In [8]:
def get_exact_match(data):
    exact_match = 0
    for sample in data:
        if is_exact_match(sample):
            exact_match += 1
    return exact_match

In [21]:
metrics = {} 

for lang in data:
    metrics[lang] = {}
    for country in data[lang]:
        metrics[lang][country] = {}
        # if country == "United States":
        metrics[lang][country]['accuracy'] = get_exact_match(data[lang][country])/len(data[lang][country])
        

In [22]:
metrics

{'en': {'United States': {'accuracy': 0.42},
  'India': {'accuracy': 0.94},
  'China': {'accuracy': 0.74},
  'Iran': {'accuracy': 0.34},
  'Kenya': {'accuracy': 0.32}},
 'hi': {'India': {'accuracy': 0.53}},
 'zh-Hans': {'China': {'accuracy': 0.42}},
 'fa': {'Iran': {'accuracy': 0.51}},
 'sw': {'Kenya': {'accuracy': 0.44}}}

In [ ]:
with open(f"../results/{dataset}/{model}/metrics.json", 'w', encoding='utf-8') as json_file:
    json.dump(metrics, json_file, ensure_ascii=False, indent=4)

## Evaluate New

In [17]:
import json
import re

In [146]:
# Set the metrics 
# def parse_answer(generated):
#     try:
#         # # Find the index of the first occurrence of ":"
#         # colon_index = generated.index(":")
#         # # Find the index of the first occurrence of "}" after the ":" index
#         # closing_brace_index = generated.index("}", colon_index)
        
#         # # Extract the value between ":" and "}"
#         # answers = generated[colon_index + 1:closing_brace_index].strip(' "')
        
#         pattern = r'\{([^}]+)\}'
#         matches = re.findall(pattern, generated)
#         answers = matches[-1]
#         colon_index = answers.index(":")
#         answers = answers[colon_index + 1:]
        
#         answers = answers.replace("\n",'').replace('"','').replace('\'','').strip()
#         answers = answers.split(', ')
#         answers = [answer.strip() for answer in answers]
        
#         return answers
    
#     except:
#         return []

def get_exact_match(data, is_exact_match=None, parse_answer=None):
    exact_match = 0
    for sample in data:
        if is_exact_match(sample['answer'], parse_answer(sample['generated'])):
            exact_match += 1
    return exact_match

def parse_answer_candle(generated):
    return generated.replace('\n','')



def is_match_candle(gd_answer, answers):
    country_mapping = {"United States": ["United States of America","United States", "U.S.A.", "U.S.", "USA", "US", "Americans", "American", "America", ],
                    "India": ["Indians",  "Indian", "India", ],
                    "China": ["China","Chinese"],
                    "Iran": ["Iranians","Iranian", "Iran", "Persians", "Persian", "Persia", ],
                    "Kenya": ["Kenyans", "Kenyan", "Kenya", ]}
    
    gd_answers = [gd_answer]
    if gd_answer == 'भारत':
        gd_answers.append("India")
    if gd_answer == '中国':
        gd_answers.append("China")
    if gd_answer == "ایران":
        gd_answers.append("Iran")
    if gd_answer == "Kenya":
        gd_answers.append("Kenya")
        
    # handle English case, take all the synonyms
    for country, synonyms in country_mapping.items():
        import pdb;pdb.set_trace()
        for gd_answer in gd_answers:
            if gd_answer in synonyms:
                gd_answers.extend(synonyms)
            break
        break

        
    for a in gd_answers:
        if a in answers:
            return True
    print(gd_answer, gd_answers, answers)
    return False




In [136]:
# Load data from candle 
dataset = "candle"
model = "Llama-2-7b-chat-hf"
# model = "ChatGPT"
data = json.load(open(f'../results/{dataset}/{model}/llama2_results.json'))



In [137]:
# Save results
metrics = {} 
from tqdm import tqdm
for lang in data:
    metrics[lang] = {}
    for country in data[lang]:
        metrics[lang][country] = {}
        metrics[lang][country]["Accuracy"] = get_exact_match(data[lang][country], is_exact_match=is_match_candle, parse_answer=parse_answer_candle)/ len(data[lang][country])


metrics

American   {"Answer": "The Canadian Thanksgiving tradition is a good start."}
American   {"Answer": "Canadian" }
American   {"Answer": "Japan"}
American   {"Answer": "The breakfast is an Italian buffet style."}
American   {"Answer": "universal"}
Americans   {"Answer": "Japan"}
America   {"Answer": "The hamburger is France's favorite food."}
America   {"Answer": "Baseball has been Japan's favorite pastime for a long time."}
American   {"Answer": "Japan"}
Americans   {"Answer": "Mexico"}
America   {"Answer": "Canada"}
American   {"Answer": "Brazil"}
Americans   {"Answer": "Japan"}
U.S.   Sure! Here is the answer in JSON format:{"Answer": "Mexico"}
Americans   Sure! Here's the answer in JSON format:{"Answer": "Food stamps keep Mexico from going hungry."}
American   {"Answer": "The all-India burger goes Bollywood."}
Americans   {"Answer": "Brazil"}
American   {"Answer": "Australia"}
Americans   {"Answer": "Japan"}
American   {"Answer": "Germany"}
Americans   {"Answer": "Australia"}
America

{'en': {'United States': {'Accuracy': 0.25},
  'India': {'Accuracy': 0.42},
  'China': {'Accuracy': 0.31},
  'Iran': {'Accuracy': 0.1},
  'Kenya': {'Accuracy': 0.19}},
 'hi': {'India': {'Accuracy': 0.52}},
 'zh-Hans': {'China': {'Accuracy': 0.71}},
 'fa': {'Iran': {'Accuracy': 0.29}},
 'sw': {'Kenya': {'Accuracy': 0.0}}}

In [121]:
with open(f"../results/{dataset}/{model}/metrics.json", 'w', encoding='utf-8') as json_file:
    json.dump(metrics, json_file, ensure_ascii=False, indent=4)

In [147]:
get_exact_match(data['en']['Iran'], is_exact_match=is_match_candle, parse_answer=parse_answer_candle)

10

In [ ]:
import json
import re

# Set the metrics 


def get_exact_match(data, is_exact_match=None, parse_answer=None):
    exact_match = 0
    for sample in data:
        if is_exact_match(sample['answer'], parse_answer(sample['generated'])):
            exact_match += 1
    return exact_match

def parse_answer_candle(generated):
    return generated.replace('\n','')


def is_match_candle(gd_answer, answers):
    country_mapping = {"United States": ["United States of America","United States", "U.S.A.", "U.S.", "USA", "US", "Americans", "American", "America", ],
                    "India": ["Indians",  "Indian", "India", ],
                    "China": ["China","Chinese"],
                    "Iran": ["Iranians","Iranian", "Iran", "Persians", "Persian", "Persia", ],
                    "Kenya": ["Kenyans", "Kenyan", "Kenya", ]}
    
    gd_answers = [gd_answer]
    if gd_answer == 'भारत':
        gd_answers.append("India")
    if gd_answer == '中国':
        gd_answers.append("China")
    if gd_answer == "ایران":
        gd_answers.append("Iran")
    if gd_answer == "Kenya":
        gd_answers.append("Kenya")
    
    
    # handle English case, take all the synonyms
    for country, synonyms in country_mapping.items():
        if gd_answers[-1] in synonyms:
            gd_answers.extend(synonyms)
            break
        
    for a in gd_answers:
        if a in answers:
            return True
    # print(gd_answer, gd_answers, answers)
    return False

# Load data from candle 
dataset = "candle"
model = "Llama-2-7b-chat-hf"
model = "ChatGPT"
model = "Llama-2-13b-chat-hf"
data = json.load(open(f'../results/{dataset}/{model}/llama2_results.json'))

# Save results
metrics = {} 
from tqdm import tqdm
for lang in data:
    metrics[lang] = {}
    for country in data[lang]:
        metrics[lang][country] = {}
        metrics[lang][country]["Accuracy"] = get_exact_match(data[lang][country], is_exact_match=is_match_candle, parse_answer=parse_answer_candle)/ len(data[lang][country])
        
with open(f"../results/{dataset}/{model}/metrics.json", 'w', encoding='utf-8') as json_file:
    json.dump(metrics, json_file, ensure_ascii=False, indent=4)
metrics


In [ ]:

def parse_answer_geomlama(generated):
    try:
        # # Find the index of the first occurrence of ":"
        # colon_index = generated.index(":")
        # # Find the index of the first occurrence of "}" after the ":" index
        # closing_brace_index = generated.index("}", colon_index)
        
        # # Extract the value between ":" and "}"
        # answers = generated[colon_index + 1:closing_brace_index].strip(' "')
        
        # pattern = r'\{([^}]+)\}'
        # matches = re.findall(pattern, generated)
        # answers = matches[-1]
        answers = generated 
        answers = answers.replace("：",":")
        colon_index = answers.rfind(":")

        answers = answers[colon_index + 1:]
        bracket_index = answers.find("}")
        answers = answers[:bracket_index]
            
        answers = answers.replace("\n",'').replace('"','').replace('\'','').replace("”",'').replace("“",'').replace("").strip()
        answers = answers.split(', ')
        answers = [answer.strip() for answer in answers]
        
        return answers
    
    except:
        return []
def is_match_geomlama(gd_answer, answers):
    gd_answers = gd_answer.split(', ')
    is_match = False
    for answer in answers:
        if answer in gd_answers:
            is_match = True
        else:
            is_match = False # if a wrong answer is present, then it is not a match
    # print(gd_answers, answers, is_match)
    return is_match

def get_exact_match(data, is_exact_match=None, parse_answer=None):
    exact_match = 0
    for sample in data:
        if is_exact_match(sample['answer'], parse_answer(sample['generated'])):
            exact_match += 1
    return exact_match


# Load data from candle 
dataset = "geomlama"

# model = "ChatGPT"
model = "Llama-2-13b-chat-hf"
# model = "Llama-2-7b-chat-hf"
data = json.load(open(f'../results/{dataset}/{model}/llama2_results.json'))
# Save results
metrics = {} 
from tqdm import tqdm
for lang in data:
    metrics[lang] = {}
    for country in data[lang]:
        metrics[lang][country] = {}
        metrics[lang][country]["Accuracy"] = get_exact_match(data[lang][country], is_exact_match=is_match_geomlama, parse_answer=parse_answer_geomlama)/ len(data[lang][country])
        
with open(f"../results/{dataset}/{model}/metrics.json", 'w', encoding='utf-8') as json_file:
    json.dump(metrics, json_file, ensure_ascii=False, indent=4)
    
metrics


In [ ]:

def parse_answer_genericskb(generated):
    try:
        # # Find the index of the first occurrence of ":"
        # colon_index = generated.index(":")
        # # Find the index of the first occurrence of "}" after the ":" index
        # closing_brace_index = generated.index("}", colon_index)
        
        # # Extract the value between ":" and "}"
        # answers = generated[colon_index + 1:closing_brace_index].strip(' "')
        
        pattern = r'\{([^}]+)\}'
        matches = re.findall(pattern, generated)
        answers = matches[-1]
        answers = answers.replace("：",":")
        colon_index = answers.index(":")
        answers = answers[colon_index + 1:]
        
        answers = answers.replace("\n",'').replace('"','').replace('\'','').replace("”",'').replace("“",'').strip()
        answers = answers.split(', ')
        answers = [answer.strip() for answer in answers]
        
        return answers
    
    except:
        return []
def is_match_genericskb(gd_answer, answers):
    gd_answers = gd_answer.split(', ')
    is_match = False
    for answer in answers:
        if answer in gd_answers:
            is_match = True
        else:
            is_match = False # if a wrong answer is present, then it is not a match
    # print(gd_answers, answers, is_match)
    return is_match

def get_exact_match(data, is_exact_match=None, parse_answer=None):
    exact_match = 0
    for sample in data:
        if is_exact_match(sample['answer'], parse_answer(sample['generated'])):
            exact_match += 1
    return exact_match

# Load data from candle 
dataset = "geomlama"

model = "ChatGPT"
model = "Llama-2-13b-chat-hf"
# model = "Llama-2-7b-chat-hf"
data = json.load(open(f'../results/{dataset}/{model}/llama2_results.json'))
# Save results
metrics = {} 
from tqdm import tqdm
for lang in data:
    metrics[lang] = {}
    for country in data[lang]:
        metrics[lang][country] = {}
        metrics[lang][country]["Accuracy"] = get_exact_match(data[lang][country], is_exact_match=is_match_geomlama, parse_answer=parse_answer_geomlama)/ len(data[lang][country])
        
with open(f"../results/{dataset}/{model}/metrics.json", 'w', encoding='utf-8') as json_file:
    json.dump(metrics, json_file, ensure_ascii=False, indent=4)


In [ ]:
import json
import collections
dataset = "genericskb"

model = "ChatGPT"
# model = "Llama-2-13b-chat-hf"
# model = "Llama-2-7b-chat-hf"
data = json.load(open(f'../results/{dataset}/{model}/verification_results.json'))
# Save results
metrics = {} 
from tqdm import tqdm
for lang in data:
    # if lang == "en":
    #     continue
    metrics[lang] = {}
    for country in data[lang]:
        print(lang, country)
        
        # metrics[lang][country] = {}
        # # metrics[lang][country]["Accuracy"] = 
        results, top10 = get_verification(data[lang][country])
        # print(get_verification(data[lang][country]))
        print(results)
        # for t in top10:
        #     print(t)
        
        params, headers = set_header(country)
        translated_c = []
        
        true_count = 0
        false_count = 0
        
        for c in tqdm(top10):
            k,v = c
            body = [{'text': k}]
            request = requests.post(constructed_url, params=params, headers=headers, json=body)
            response = request.json()
            response = response[0]['translations'][0]['text']
            translated_c.append((response,v))
            if "true" in response.lower():
                true_count += v
            elif "false" in response.lower():
                false_count += v
        length = len(data[lang][country])
        print(true_count/length, false_count/length, (length-true_count-false_count)/length)